In [ ]:
import numpy as np
from datetime import date
import matplotlib
import matplotlib.pyplot as plt

from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u


In [ ]:
def AzElPrime(Az, El, delta, alpha):
    # First find ElPrime
    sinElPrime = np.cos(alpha - El) * np.sin(delta) * np.cos(El) - np.cos(delta) * np.sin(El)
    ElPrime = Angle(-np.arcsin(sinElPrime))
    # Now find beta
    cosBeta = (np.cos(delta) - np.sin(El) * np.sin(ElPrime)) / (np.cos(El) * np.cos(ElPrime))
    beta = Angle(np.arccos(cosBeta))
    #Now find AzPrime by Newton's method
    cosAzPrime = np.cos(Az)
    AzError = 100.0
    while AzError > 1E-6:
        sinAzPrime = np.sqrt(1.0 - cosAzPrime**2)

        num = np.cos(Az) - cosAzPrime * np.cos(beta) + np.sin(ElPrime) * sinAzPrime * np.sin(beta)
        den = np.cos(Az) + sinAzPrime * np.cos(beta) + np.sin(ElPrime) * cosAzPrime * np.sin(beta)
        nextCosAzPrime = cosAzPrime + num / den
        AzError = abs(nextCosAzPrime - cosAzPrime)
        cosAzPrime = nextCosAzPrime
    AzPrime = Angle(np.arccos(cosAzPrime))
    return [AzPrime.deg, ElPrime.deg]

In [ ]:
deltaAz0 = 4.25
deltaEl0 = 0.15
Az = Angle(45.0 * u.deg)
delta = Angle(0.08 * u.deg)
alpha = Angle(20.0 * u.deg)
els = np.linspace(20, 87, 100)
modEls = Angle(els * u.deg)
modDeltaEls = []
modDeltaAzs = []
for El in modEls:
    [AzPrime, ElPrime] = AzElPrime(Az, El, delta, alpha)
    modDeltaEls.append(El.deg - ElPrime + deltaEl0)
    modDeltaAzs.append(Az.deg - AzPrime + deltaAz0)
plt.subplot(1,2,1)
plt.plot(els, modDeltaAzs, color='green', ls='--', lw=3)
plt.subplot(1,2,2)
plt.plot(els, modDeltaEls, color='green', ls='--', lw=3)

In [ ]:
dir = "/Users/cslage/Research/LSST/code/starTracker/tab_delimited_summaries"

In [ ]:
date = 28
camera = 'Wide'
num = 101
filename = f"StarTracker_Summary_{camera}_202211{date}.txt"
data = np.loadtxt(dir+"/"+filename, skiprows=1)
Azs = []
Els = []
deltaAzs = []
deltaEls = []
for j in range(data.shape[0]):
    [seqNum,raStart,decStart,raSolve,decSolve,azStart,elStart,az,\
     el,deltaAz,deltaEl,rot,rms_error] = data[j]
    Azs.append(azStart)
    Els.append(elStart)
    deltaAzs.append(deltaAz)
    deltaEls.append(deltaEl)
Azs = np.array(Azs)
Els = np.array(Els)
deltaAzs = np.array(deltaAzs)
deltaEls = np.array(deltaEls)


In [ ]:
plt.figure(figsize=(8,4))
plt.subplots_adjust(wspace=0.5, hspace=0.5)
plt.subplot(1,2,1)
plt.title("DeltaAz vs Alt")
plt.plot(els, modDeltaAzs, color='green', ls='--', lw=3)
plt.scatter(Els, deltaAzs)
plt.text(5.0, 5.0, f"Axis tilt = {delta.deg*60.0} arcminutes")
plt.xlim(0,90)
plt.xlabel("Altitude(degrees)")
plt.ylabel("DeltaAz(degrees)")
plt.subplot(1,2,2)
plt.title("DeltaAlt vs Alt")
plt.plot(els, modDeltaEls, color='green', ls='--', lw=3)
plt.scatter(Els, deltaEls)
plt.xlim(0,90)
plt.xlabel("Altitude(degrees)")
plt.ylabel("DeltaAlt(degrees)")
plt.savefig(dir+"/Deltas_vs_AltAz_01Dec22.png")